In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import cv2

from camera import Camera
import structure
import processor
import features

In [2]:
img1 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0289.JPG')
img2 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0288.JPG')
img3 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0287.JPG')
print("Bilder geladen")

Bilder geladen


In [3]:
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(
    cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), None)
kp2, des2 = sift.detectAndCompute(
    cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), None)
kp3, des3 = sift.detectAndCompute(
    cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY), None)

kp1 = np.array([n.pt for n in kp1])
kp2 = np.array([n.pt for n in kp2])
kp3 = np.array([n.pt for n in kp3])


In [4]:
def matchPoints(kp1,des1,kp2,des2):
    # Find point matches
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=100)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Apply Lowe's SIFT matching ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)

    paare = np.array([[m.queryIdx, m.trainIdx] for m in good])
    
    # Constrain matches to fit homography
    retval, mask = cv2.findHomography(
        kp1[paare[:, 0]], kp2[paare[:, 1]], cv2.RANSAC, 100.0)
    mask = mask.ravel()

    # We select only inlier points
    paare = paare[mask == 1]
    return paare

In [5]:
paare = matchPoints(kp1, des1, kp2, des2)
paare2 = matchPoints(kp2[paare[:, 1]], des2[paare[:, 1]], kp3, des3)


In [6]:
pts1 = kp1[paare[:,0]].T
pts2 = kp2[paare[:,1]].T
pts3 = kp3[paare2[:,1]].T

In [7]:
points1 = processor.cart2hom(pts1)
points2 = processor.cart2hom(pts2)
points3 = processor.cart2hom(pts3)


In [238]:
fig, ax = plt.subplots(1, 3)
ax[0].autoscale_view('tight')
ax[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
ax[0].plot(points1[0], points1[1], 'r.')
ax[1].autoscale_view('tight')
ax[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[1].plot(points2[0], points2[1], 'r.')
ax[2].autoscale_view('tight')
ax[2].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
ax[2].plot(kp3[paare2[:, 1], 0], kp3[paare2[:, 1], 1], 'r.')
fig.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [9]:
height, width, ch = img1.shape
intrinsic = np.array([  # for dino
    [3030.65, 0, width / 2-6],
    [0, 3030.65, height / 2+17],
    [0, 0, 1]])
intrinsic

array([[3.03065e+03, 0.00000e+00, 1.99400e+03],
       [0.00000e+00, 3.03065e+03, 1.51700e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00]])

In [10]:
def scale_and_translate_points(points):
    """ Scale and translate image points so that centroid of the points
        are at the origin and avg distance to the origin is equal to sqrt(2).
    :param points: array of homogenous point (3 x n)
    :returns: array of same input shape and its normalization matrix
    """
    x = points[0]
    y = points[1]
    center = points.mean(axis=1)  # mean of each row
    cx = x - center[0]  # center the points
    cy = y - center[1]
    dist = np.sqrt(np.power(cx, 2) + np.power(cy, 2))
    scale = np.sqrt(2) / dist.mean()
    norm3d = np.array([
        [scale, 0, -scale * center[0]],
        [0, scale, -scale * center[1]],
        [0, 0, 1]
    ])

    return np.dot(norm3d, points), norm3d


In [11]:
def correspondence_matrix(p1, p2):
    """Each row in the A matrix below is constructed as
        [x'*x, x'*y, x', y'*x, y'*y, y', x, y, 1]"""
    p1x, p1y = p1[:2]
    p2x, p2y = p2[:2]

    return np.array([
        p1x * p2x, p1x * p2y, p1x,
        p1y * p2x, p1y * p2y, p1y,
        p2x, p2y, np.ones(len(p1x))
    ]).T


In [12]:
def compute_essential_normalized(p1, p2):
    """ Computes the fundamental or essential matrix from corresponding points
        using the normalized 8 point algorithm.
    :input p1, p2: corresponding points with shape 3 x n
    :returns: fundamental or essential matrix with shape 3 x 3
    """
    n = p1.shape[1]
    if p2.shape[1] != n:
        raise ValueError('Number of points do not match.')

    # preprocess image coordinates
    p1n, T1 = scale_and_translate_points(p1)
    p2n, T2 = scale_and_translate_points(p2)

    # compute F or E with the coordinates
    A = correspondence_matrix(p1n, p2n)
    # compute linear least square solution
    U, S, V = np.linalg.svd(A)
    F = V[-1].reshape(3, 3)

    # constrain F. Make rank 2 by zeroing out last singular value
    U, S, V = np.linalg.svd(F)
    S[-1] = 0
    S = [1, 1, 0]  # Force rank 2 and equal eigenvalues
    F = U @ np.diag(S) @ V

    # reverse preprocessing of coordinates
    # We know that P1' E P2 = 0
    F = T1.T@F@T2

    return F / F[2, 2]


In [13]:
# Calculate essential matrix with 2d points.
# Result will be up to a scale
# First, normalize points
points1n = np.dot(np.linalg.inv(intrinsic), points1)
points2n = np.dot(np.linalg.inv(intrinsic), points2)
#cv2.undistortPoints(pts1, intrinsic, None)[:,0,:].T

In [14]:
E = compute_essential_normalized(points1n, points2n)
#E1 = structure.compute_essential_normalized(points1n, points2an)
print('Computed essential matrix:', (-E / E[0][1]))


Computed essential matrix: [[-0.03157542 -1.         -0.23135702]
 [ 0.95870465 -0.01264242 -0.00189078]
 [ 0.25888847  0.20644202  0.04803392]]


In [15]:
P1 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
P1

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]])

In [16]:
_,R,t,_= cv2.recoverPose(E,pts1.T,pts2.T, intrinsic)
R = np.linalg.inv(R)
t = -R@t
P2 = np.c_[R,t]
P2


array([[ 0.97981755,  0.04640084, -0.19443387, -0.00105049],
       [-0.04064291,  0.99861194,  0.03350133, -0.22537157],
       [ 0.19571847, -0.02492283,  0.98034337,  0.97427232]])

In [17]:
#tripoints3d = structure.reconstruct_points(points1n, points2n, P1, P2)
tripoints3d = structure.linear_triangulation(
    points1n, points2n, P1, P2)


In [18]:
%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(tripoints3d[0], tripoints3d[1], tripoints3d[2], 'r.')
ax.plot([0], [0], [0], 'g.')
ax.plot(-P2[0, 3], -P2[1, 3], -P2[2, 3], 'g.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
#plt.axis('square')
plt.show()

QSocketNotifier: Can only be used with threads started with QThread
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [19]:
tripoints3d[:,paare2[:,0]].T

array([[-1.01237862, -0.2723046 ,  1.5527952 ,  1.        ],
       [-0.97998248, -0.45032163,  1.5206182 ,  1.        ],
       [-0.97333944, -0.492564  ,  1.51477244,  1.        ],
       ...,
       [ 0.86804847, -0.14806936,  1.32003067,  1.        ],
       [ 0.87509442, -0.03954603,  1.33156662,  1.        ],
       [ 0.86850577, -0.1444281 ,  1.31957639,  1.        ]])

In [20]:
retval, r2,t2,_ = cv2.solvePnPRansac(tripoints3d[:3,paare2[:,0]].T, kp3[paare2[:,1]], intrinsic, None)
retval, r2,t2

(True,
 array([[0.23347676],
        [0.29347857],
        [0.04032486]]),
 array([[-0.9576606 ],
        [-0.21836462],
        [ 0.91359771]]))

In [21]:
R2,_ = cv2.Rodrigues(r2)
P3 = np.c_[R2,t2]
P3

array([[ 0.95663986, -0.00551953,  0.29122108, -0.9576606 ],
       [ 0.07323144,  0.97226245, -0.22213257, -0.21836462],
       [-0.28191726,  0.23382741,  0.93050921,  0.91359771]])

In [22]:
points3n = np.dot(np.linalg.inv(intrinsic), points3)
points3n


array([[-0.58855988, -0.59437814, -0.5953235 , ...,  0.13811134,
         0.14129843,  0.13844267],
       [-0.36082361, -0.43854992, -0.45736602, ..., -0.31761943,
        -0.25845193, -0.31562222],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ]])

In [23]:
tripoints3d2 = structure.linear_triangulation(
    points2n.T[paare2[:, 0]].T, points3n, P2, P3)
tripoints3d2.shape

(4, 651)

In [24]:
%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(tripoints3d[0], tripoints3d[1], tripoints3d[2], 'r.')
ax.plot(tripoints3d2[0], tripoints3d2[1], tripoints3d2[2], 'b.')
ax.plot([0], [0], [0], 'g.')
ax.plot(-P2[0, 3], -P2[1, 3], -P2[2, 3], 'g.')
ax.plot(-t2[0], -t2[1], -t2[2], 'g.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
plt.axis('square')
ax.set_ylim([-1, 1])
ax.set_xlim([-1, 1])
ax.set_zlim([-1, 2])
plt.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


## Some more points

In [25]:
paare3 = matchPoints(kp2, des2, kp3, des3)


In [26]:
pts2_3 = kp2[paare3[:, 0]].T
pts3_2 = kp3[paare3[:, 1]].T
points2_3 = processor.cart2hom(pts2_3)
points3_2 = processor.cart2hom(pts3_2)
points2_3n = np.dot(np.linalg.inv(intrinsic), points2_3)
points3_2n = np.dot(np.linalg.inv(intrinsic), points3_2)


In [27]:
fig, ax = plt.subplots(1, 3)
ax[0].autoscale_view('tight')
ax[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
ax[0].plot(points1[0], points1[1], 'r.')
ax[1].autoscale_view('tight')
ax[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[1].plot(points2[0], points2[1], 'r.')
ax[2].autoscale_view('tight')
ax[2].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
ax[2].plot(kp3[paare2[:, 1], 0], kp3[paare2[:, 1], 1], 'r.')
ax[1].plot(points2_3[0], points2_3[1], 'b.')
ax[2].plot(points3_2[0], points3_2[1], 'b.')
fig.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [28]:
tripoints3d2 = structure.linear_triangulation(
    points2_3n, points3_2n, P2, P3)
tripoints3d2.shape

(4, 1653)

In [29]:
%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(tripoints3d[0], tripoints3d[1], tripoints3d[2], 'r.')
ax.plot(tripoints3d2[0], tripoints3d2[1], tripoints3d2[2], 'b.')
ax.plot([0], [0], [0], 'g.')
ax.plot(-P2[0, 3], -P2[1, 3], -P2[2, 3], 'g.')
ax.plot(-t2[0], -t2[1], -t2[2], 'g.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
plt.axis('square')
ax.set_ylim([-1, 1])
ax.set_xlim([-1, 1])
ax.set_zlim([-1, 2])
plt.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [30]:
img4 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0286.JPG')
kp4, des4 = sift.detectAndCompute(
    cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY), None)
kp4 = np.array([n.pt for n in kp4])

paare4 = matchPoints(kp3, des3, kp4, des4)

In [31]:

pts3_4 = kp3[paare4[:, 0]].T
pts4_3 = kp4[paare4[:, 1]].T
points3_4 = processor.cart2hom(pts3_4)
points4_3 = processor.cart2hom(pts4_3)
points3_4n = np.dot(np.linalg.inv(intrinsic), points3_4)
points4_3n = np.dot(np.linalg.inv(intrinsic), points4_3)


In [32]:
fig, ax = plt.subplots(1, 2)
ax[0].autoscale_view('tight')
ax[0].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
ax[0].plot(points3_4[0], points3_4[1], 'r.')
ax[1].autoscale_view('tight')
ax[1].imshow(cv2.cvtColor(img4, cv2.COLOR_BGR2RGB))
ax[1].plot(points4_3[0], points4_3[1], 'r.')
fig.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


## Bundle Adjustment

In [130]:
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix

In [34]:
r1,_ = cv2.Rodrigues(np.float32(P1[:3, :3]))
t1 = P1[:,3]
r2, _ = cv2.Rodrigues(np.float32(P2[:3, :3]))
t2 = P2[:, 3]
r3, _ = cv2.Rodrigues(np.float32(P3[:3, :3]))
t3 = P3[:, 3]
fx = intrinsic[0,0]
fy = intrinsic[1,1]
cx = intrinsic[0,2]
cy = intrinsic[1,2]

In [188]:
tripoints3d.shape

(4, 3718)

In [176]:
x0 = np.hstack((fx,fy,cx,cy,r1.ravel(),t1.ravel(),r2.ravel(),t2.ravel(),r3.ravel(),t3.ravel(),tripoints3d[:3,:].T.ravel()))

In [178]:
l = np.hstack((pts1.T.ravel(),pts2.T.ravel(),pts3.T.ravel()))

In [192]:
pts3.T.shape

(651, 2)

In [194]:
(3718*2+651)*2

16174

In [180]:
def project(x0):
    fx = x0[0]
    fy = x0[1]
    cx = x0[2]
    cy = x0[3]
    
    K = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]])

    r1 = x0[4:7]
    t1 = x0[7:10]
    r2 = x0[10:13]
    t2 = x0[13:16]
    r3 = x0[16:19]
    t3 = x0[19:22]

    coords = x0[22:]
    coords = coords.reshape(len(coords)//3,3)

    p1,_ = cv2.projectPoints(coords, r1,t1,K,None)
    p2, _ = cv2.projectPoints(coords, r2, t2, K, None)
    p3, _ = cv2.projectPoints(coords[paare2[:, 0]], r3, t3, K, None)

    p = np.hstack((p1.ravel(),p2.ravel(),p3.ravel()))
    return p-l


In [203]:
A = lil_matrix((len(l), len(x0)), dtype=int)
print(A.shape)
A[:, :4] = 1
n1 = len(pts1.T)
n3 = len(paare2[:, 0])
for i in range(n1):
    A[2*i:2*i+2, 4:10] = 1
    A[2*i:2*i+2, 22+i*3:25+i*3] = 1

maxy = 0
for i in range(n1):
    A[2*(n1+i):2*(n1+i)+2, 10:16] = 1
    A[2*(n1+i):2*(n1+i)+2, 22+i*3:25+i*3] = 1
    maxy = 25+i*3
print(maxy)

max = 0
for i in range(n3):
    v = n1 + n1
    A[2*(v+i):2*(v+i)+2, 16:22] = 1
    z = paare2[i, 0]
    A[2*(v+i):2*(v+i)+2, 22+z*3:25+z*3] = 1
    max = 2*(v+i)+1
print(max)


(16174, 11176)
11176
16173


In [209]:

list(A.toarray()[:5,:30])

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1])]

In [216]:
res = least_squares(project, x0, jac_sparsity=A, verbose=2,
                    x_scale='jac', method='trf', ftol=1e-3)


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.2834e+05                                    3.82e+07    
       1              2         3.6074e+05      1.68e+05       8.32e+01       9.79e+06    
       2              3         2.4338e+05      1.17e+05       3.96e+01       1.79e+07    
       3              4         1.3875e+05      1.05e+05       1.14e+01       5.03e+07    
       4              5         6.1956e+04      7.68e+04       2.47e+01       1.14e+07    
       5              6         4.9780e+04      1.22e+04       2.48e+01       3.13e+06    
       6              7         3.6370e+04      1.34e+04       2.71e+01       4.75e+05    
       7             10         3.5979e+04      3.91e+02       2.24e+00       1.15e+06    
       8             11         3.4479e+04      1.50e+03       1.02e+01       3.23e+05    
       9             12         3.4303e+04      1.76e+02       6.72e+00       1.46e+06    

In [221]:
fx = res.x[0]
fy = res.x[1]
cx = res.x[2]
cy = res.x[3]

K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
K

array([[3.01410725e+03, 0.00000000e+00, 1.98401799e+03],
       [0.00000000e+00, 3.12279416e+03, 1.52288083e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [222]:
intrinsic

array([[3.03065e+03, 0.00000e+00, 1.99400e+03],
       [0.00000e+00, 3.03065e+03, 1.51700e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00]])

In [219]:
project(res.x)

array([-0.01447455,  0.15881271,  0.00168593, ...,  0.41019293,
        0.20103729,  0.07919818])

In [224]:
project(x0)


array([ 0.12965523, -8.40143123, -0.82013417, ...,  2.6961788 ,
        0.60731602, -0.56348183])

In [236]:
coords_new = res.x[22:]
coords_new = coords_new.reshape(len(coords_new)//3,3)

t1 = res.x[7:10]
t2 = res.x[13:16]
t3 = res.x[19:22]
t1

array([-0.03815566,  0.02402933, -0.15801609])

In [232]:
coords_new.T

array([[-1.00553646, -1.01953347, -1.00409616, ...,  0.87509442,
         0.89586751,  0.86850577],
       [-0.42699544, -0.27009344, -0.40814642, ..., -0.03954603,
         0.23656618, -0.1444281 ],
       [ 1.53287928,  1.5548741 ,  1.53279683, ...,  1.33156662,
         1.3647532 ,  1.31957639]])

In [237]:
%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(coords_new.T[0], coords_new.T[1], coords_new.T[2], 'r.')
ax.plot(-t1[0], -t1[1], -t1[2], 'g.')
ax.plot(-t2[0], -t2[1], -t2[2], 'g.')
ax.plot(-t3[0], -t3[1], -t3[2], 'g.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
plt.axis('square')
ax.set_ylim([-1, 2])
ax.set_xlim([-1, 2])
ax.set_zlim([-1, 2])
plt.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
